### Overview
This notebook imports the extracted data from the unannotated scRNA-seq (publicly available dataset [2]) and reference to perform single-cell annotation using scmap [3]. 

**This notebook is written in R.**

In [1]:
.libPaths('/home/chiacmm/rpackages')

In [ ]:
#Libraries and global setting

library(SingleCellExperiment)
library(scmap)
library(scran)
library(base) #it failed without loading base once on rpy2env
library(tools)
library("data.table")

# Seed
set.seed(4) #For reproducibility

### Steps performed

1. Import the query data (unannotated scRNA-seq)

2. Import the reference data (to be used in annotating the query data)

3. Convert query and reference data to SingleCellExperiment format

4. scmap: 
- set the HVG as features
- calculate indices for each cell in reference
- project reference to query set
- cluster-annotate query cells

5. Export annotation results

#### Import the query data (unannotated scRNA-seq)

In [3]:
#Import query data
fullpath_data_que_geneexpr <- "../../data/demo_public/output/que_exprs.csv"
fullpath_data_que_cellID <- "../../data/demo_public/output/que_lcellID.csv"
fullpath_data_que_genes <- "../../data/demo_public/output/que_lgenes.csv"
fullpath_data_que_hvg_genes <- "../../data/demo_public/output/que_l_hvg_genes.csv"

#Read query data
df_geneexprs_nomlog_que <- fread(file=fullpath_data_que_geneexpr, header=TRUE, verbose=TRUE)
l_cellID_que <- read.csv(fullpath_data_que_cellID,stringsAsFactors = F,header=TRUE,fill = F)
l_genes_que <- read.csv(fullpath_data_que_genes,stringsAsFactors = F,header=TRUE,fill = F)
l_hvggenes_que <- read.csv(fullpath_data_que_hvg_genes,stringsAsFactors = F,header=TRUE,fill = F)

  OpenMP version (_OPENMP)       201511
  omp_get_num_procs()            16
  R_DATATABLE_NUM_PROCS_PERCENT  unset (default 50)
  R_DATATABLE_NUM_THREADS        unset
  R_DATATABLE_THROTTLE           unset (default 1024)
  omp_get_thread_limit()         2147483647
  omp_get_max_threads()          16
  OMP_THREAD_LIMIT               unset
  OMP_NUM_THREADS                unset
  RestoreAfterFork               true
  data.table is using 8 threads with throttle==1024. See ?setDTthreads.
Input contains no \n. Taking this to be a filename to open
[01] Check arguments
  Using 8 threads (omp_get_max_threads()=16, nth=8)
  NAstrings = [<<NA>>]
  None of the NAstrings look like numbers.
  show progress = 0
  0/1 column will be read as integer
[02] Opening the file
  Opening file ../../data/demo_public/output/que_exprs.csv
  File opened, size = 576MB (604675621 bytes).
  Memory mapped ok
[03] Detect and skip BOM
[04] Arrange mmap to be \0 terminated
  \n has been found in the input and different

#### Import the reference data (to be used in annotating the query data)

In [4]:
#Import reference data
fullpath_data_ref_geneexpr <- "../../data/demo_public/output/ref_exprs.csv"
fullpath_data_ref_cellID <- "../../data/demo_public/output/ref_lcellID.csv"
fullpath_data_ref_genes <- "../../data/demo_public/output/ref_lgenes.csv"
fullpath_data_ref_hvg_genes <- "../../data/demo_public/output/ref_l_hvg_genes.csv"
fullpath_data_ref_celltype <- "../../data/demo_public/output/ref_lcelltype.csv"

#Read reference data
df_geneexprs_nomlog_ref <- fread(file=fullpath_data_ref_geneexpr, header=TRUE, verbose=TRUE)
l_cellID_ref <- read.csv(fullpath_data_ref_cellID,stringsAsFactors = F,header=TRUE,fill = F)
l_genes_ref <- read.csv(fullpath_data_ref_genes,stringsAsFactors = F,header=TRUE,fill = F)
l_hvggenes_ref <- read.csv(fullpath_data_ref_hvg_genes,stringsAsFactors = F,header=TRUE,fill = F)
l_celltype_ref <- read.csv(fullpath_data_ref_celltype,stringsAsFactors = F,header=TRUE,fill = F)

  OpenMP version (_OPENMP)       201511
  omp_get_num_procs()            16
  R_DATATABLE_NUM_PROCS_PERCENT  unset (default 50)
  R_DATATABLE_NUM_THREADS        unset
  R_DATATABLE_THROTTLE           unset (default 1024)
  omp_get_thread_limit()         2147483647
  omp_get_max_threads()          16
  OMP_THREAD_LIMIT               unset
  OMP_NUM_THREADS                unset
  RestoreAfterFork               true
  data.table is using 8 threads with throttle==1024. See ?setDTthreads.
Input contains no \n. Taking this to be a filename to open
[01] Check arguments
  Using 8 threads (omp_get_max_threads()=16, nth=8)
  NAstrings = [<<NA>>]
  None of the NAstrings look like numbers.
  show progress = 0
  0/1 column will be read as integer
[02] Opening the file
  Opening file ../../data/demo_public/output/ref_exprs.csv
  File opened, size = 576MB (604364453 bytes).
  Memory mapped ok
[03] Detect and skip BOM
[04] Arrange mmap to be \0 terminated
  \n has been found in the input and different

#### Convert query and reference data to SingleCellExperiment format

In [5]:
#Create SingleCellExperiment Query
rownames(df_geneexprs_nomlog_que) <- l_genes_que[,1]
colnames(df_geneexprs_nomlog_que) <- l_cellID_que[,1]
sce_que <- SingleCellExperiment(list(logcounts=df_geneexprs_nomlog_que))

#Use gene names as feature symbols
rowData(sce_que)$feature_symbol <- rownames(sce_que)

In [6]:
#Create SingleCellExperiment Reference # 
rownames(df_geneexprs_nomlog_ref) <- l_genes_ref[,1]
colnames(df_geneexprs_nomlog_ref) <- l_cellID_ref[,1]
sce_ref <- SingleCellExperiment(list(logcounts=df_geneexprs_nomlog_ref))

#Use gene names as feature symbols
rowData(sce_ref)$feature_symbol <- rownames(sce_ref)

#Add celltype labels
l_list_celltype_ref <- lapply(strsplit(as.character(l_celltype_ref[,1]),split=','),trimws)
colData(sce_ref)$cell_type1 <- l_list_celltype_ref

#### scmap begins - set the HVG as features

In [7]:
# Set Features
sce_que <- setFeatures(sce_que, features=c(l_hvggenes_que[,1])) 
#sce_ref <- setFeatures(sce_ref, features=l_hvggenes_ref) #Raised Error in !unlist(lapply(subcentroids, is.null)): invalid argument type
sce_ref <- setFeatures(sce_ref, features=c(l_hvggenes_ref[,1])) 
sce_que
sce_ref

class: SingleCellExperiment 
dim: 31053 4053 
metadata(0):
assays(1): logcounts
rownames(31053): Xkr4 Gm1992 ... Vmn2r122 CAAA01147332.1
rowData names(2): feature_symbol scmap_features
colnames(4053): TGCCAAATCAAAGACA-L8TX_190312_01_C01
  TTGTAGGCATGAACCT-L8TX_190312_01_E01 ...
  TGCGGGTAGTTACGGG-L8TX_190312_01_H01
  TTGCGTCCACACTGCG-L8TX_180907_01_F11
colData names(0):
reducedDimNames(0):
mainExpName: NULL
altExpNames(0):

class: SingleCellExperiment 
dim: 31053 4052 
metadata(0):
assays(1): logcounts
rownames(31053): Xkr4 Gm1992 ... Vmn2r122 CAAA01147332.1
rowData names(2): feature_symbol scmap_features
colnames(4052): GGCTCGAAGGAGTTGC-L8TX_180907_01_F11
  GGACATTCAGTACACT-L8TX_180907_01_E11 ...
  GGAATAAAGTTTCCTT-L8TX_180907_01_F11
  CTCGGGATCGCATGGC-L8TX_180907_01_F11
colData names(1): cell_type1
reducedDimNames(0):
mainExpName: NULL
altExpNames(0):

#### Calculate indices for each cell in reference

In [8]:
#In the scmap-cell index is created by a product quantiser algorithm in a way that every cell in the reference is identified with a set of sub-centroids found via k-means clustering based on a subset of the features.
#scmap-cell index consists of two items: ## [1] "subcentroids" "subclusters"
system.time(sce_ref <- indexCell(sce_ref))
dim(metadata(sce_ref)$scmap_cell_index$subcentroids[[1]])
length(metadata(sce_ref)$scmap_cell_index$subcentroids) 
dim(metadata(sce_ref)$scmap_cell_index$subclusters)

Parameter M was not provided, will use M = n_features / 10 (if n_features <= 1000), where n_features is the number of selected features, and M = 100 otherwise.

Parameter k was not provided, will use k = sqrt(number_of_cells)



   user  system elapsed 
  5.876   0.154   6.027 

[1] 20 63

[1] 100

[1]  100 4052

#### Projection of reference to query set

In [9]:
#scmapCell_results contains results of projection for each reference dataset in a list:
#The cells matrix contains the top 10 (scmap default) cell IDs of the cells of the reference dataset that a given cell of the projection dataset is closest to:
#The similarities matrix contains corresponding cosine similarities:
system.time(scmapCell_results <- scmapCell(projection=sce_que, index_list=list(Allen=metadata(sce_ref)$scmap_cell_index)))
dim(scmapCell_results$Allen$cells)
df_scmapCell_results <- data.frame(scmapCell_results$Allen$cells)
dim(df_scmapCell_results)

   user  system elapsed 
 34.881   0.086  34.948 

[1]   10 4053

[1]   10 4053

#### Cluster-annotate query cells

In [10]:
system.time(scmapCell_clusters <- scmapCell2Cluster(scmapCell_results=scmapCell_results, cluster_list=list(as.character(colData(sce_ref)$cell_type1))))
dim(scmapCell_clusters$scmap_cluster_labs)
dim(scmapCell_clusters$scmap_cluster_siml)
length(scmapCell_clusters$combined_labs)
unique(unlist(scmapCell_clusters$scmap_cluster_labs, use.names = FALSE))

   user  system elapsed 
  0.054   0.000   0.054 

[1] 4053    1

[1] 4053    1

[1] 4053

Allen
Glutamatergic
Non-Neuronal
GABAergic
unassigned


#### Export annotation results

In [11]:
# Write data
#convert list of lists to data frame
df_scmapCell_clusters <- data.frame(l_cellID_que[,1], scmapCell_clusters$scmap_cluster_labs[,1], scmapCell_clusters$scmap_cluster_siml[,1],scmapCell_clusters$combined_labs)
names(df_scmapCell_clusters) <- c("cellID", "annotation","scmap_cluster_siml","scmap_combined_labs")

fullpath_output <- "../../data/demo_public/output/scmap_annotation.csv"
system.time(fwrite(df_scmapCell_clusters, fullpath_output))

   user  system elapsed 
  0.002   0.000   0.005 

In [12]:
sessionInfo()

R version 4.1.3 (2022-03-10)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Debian GNU/Linux 10 (buster)

Matrix products: default
BLAS/LAPACK: /home/chiacmm/.conda/envs/findsyn/lib/libopenblasp-r0.3.21.so

locale:
 [1] LC_CTYPE=en_US       LC_NUMERIC=C         LC_TIME=en_US       
 [4] LC_COLLATE=en_US     LC_MONETARY=en_US    LC_MESSAGES=en_US   
 [7] LC_PAPER=en_US       LC_NAME=C            LC_ADDRESS=C        
[10] LC_TELEPHONE=C       LC_MEASUREMENT=en_US LC_IDENTIFICATION=C 

attached base packages:
[1] tools     stats4    stats     graphics  grDevices utils     datasets 
[8] methods   base     

other attached packages:
 [1] data.table_1.14.6           scran_1.22.1               
 [3] scuttle_1.4.0               scmap_1.16.0               
 [5] SingleCellExperiment_1.16.0 SummarizedExperiment_1.24.0
 [7] Biobase_2.54.0              GenomicRanges_1.46.1       
 [9] GenomeInfoDb_1.30.1         IRanges_2.28.0             
[11] S4Vectors_0.32.4            BiocGenerics_0.4

##### Reference
1. Chia, C. M., Roig Adam, A., & Moro, A. (2022). *In silico* multiple single-subject neural tissue screening using deconvolution on pseudo-bulk RNA-seq - a prototype. Bioinformatics and Systems Biology joint degree program. Vrije Universiteit Amsterdam and University of Amsterdam. 

2. Allen Institute for Brain Science (2004). Allen Mouse Brain Atlas, Mouse Whole Cortex and Hippocampus 10x. Available from mouse.brain-map.org. Allen Institute for Brain Science (2011).

3. Kiselev, V. Y., Yiu, A., & Hemberg, M. (2018). scmap: projection of single-cell RNA-seq data across data sets. Nature methods, 15(5), 359–362. https://doi.org/10.1038/nmeth.4644